<a href="https://colab.research.google.com/github/sokrypton/ColabDesign/blob/v1.1.1/af/examples/af_pseudo_diffusion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#AF_pseudo_diffusion + proteinMPNN
Hacking AlphaFold to be a diffusion model (for backbone generation). At each step unconditional logits from proteinMPNN are added.

In [1]:
#@title setup
%%time
import os
if not os.path.isdir("params"):
  # get code
  os.system("pip -q install git+https://github.com/sokrypton/ColabDesign.git@v1.1.1")
  # for debugging
  os.system("ln -s /usr/local/lib/python3.7/dist-packages/colabdesign colabdesign")
  # download params
  os.system("mkdir params")
  os.system("apt-get install aria2 -qq")
  os.system("aria2c -q -x 16 https://storage.googleapis.com/alphafold/alphafold_params_2022-03-02.tar")
  os.system("tar -xf alphafold_params_2022-03-02.tar -C params")

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import os
from colabdesign import mk_afdesign_model, clear_mem
from colabdesign.mpnn import mk_mpnn_model
from IPython.display import HTML
from google.colab import files
import numpy as np

CPU times: user 1.57 s, sys: 323 ms, total: 1.89 s
Wall time: 1min 47s


In [60]:
clear_mem()
af_model = mk_afdesign_model(protocol="hallucination", use_templates=True)
af_model.prep_inputs(length=100)
mpnn_model = mk_mpnn_model()

print("length",af_model._len)
print("weights",af_model.opt["weights"])

length 100
weights {'con': 1.0, 'exp_res': 0.0, 'helix': 0.0, 'pae': 0.0, 'plddt': 0.0, 'seq_ent': 0.0}


In [67]:
af_model.restart(mode="gumbel")
af_model._inputs["batch"] = {"aatype":np.full(af_model._len,0),
                             "all_atom_mask":np.tile(np.arange(37)[None] < 4, (af_model._len,1)).astype(float),
                             "all_atom_positions":np.tile(5.5 * np.arange(af_model._len)[:,None,None], (1,37,3))}
for k in range(100):
  # add noise
  w = 1 - k/100
  random_noise_xyz = np.random.normal(size=(af_model._len,37,3)) * w
  af_model._inputs["batch"]["all_atom_positions"] += random_noise_xyz

  # denoise
  aux = af_model.predict(return_aux=True, verbose=False)
  af_model._inputs["batch"]["all_atom_positions"] = af_model.aux["atom_positions"]

  # sample a new sequence with proteinMPNN for the given backbone (decrease temperature at each stage)
  mpnn_model.get_af_inputs(af_model)
  af_model._params["seq"] = 0.9 * af_model._params["seq"] + 0.1 * mpnn_model.get_unconditional_logits()[...,:20]

  # save results
  af_model._save_results(aux)
  af_model._k += 1

1 models [0] recycles 0 hard 1 soft 0 temp 1 loss 3.63 con 3.63 plddt 0.38 ptm 0.30
2 models [0] recycles 0 hard 1 soft 0 temp 1 loss 3.29 con 3.29 plddt 0.49 ptm 0.40
3 models [0] recycles 0 hard 1 soft 0 temp 1 loss 3.02 con 3.02 plddt 0.57 ptm 0.49
4 models [0] recycles 0 hard 1 soft 0 temp 1 loss 2.74 con 2.74 plddt 0.65 ptm 0.56
5 models [0] recycles 0 hard 1 soft 0 temp 1 loss 2.79 con 2.79 plddt 0.63 ptm 0.55
6 models [0] recycles 0 hard 1 soft 0 temp 1 loss 2.69 con 2.69 plddt 0.66 ptm 0.55
7 models [0] recycles 0 hard 1 soft 0 temp 1 loss 2.19 con 2.19 plddt 0.79 ptm 0.69
8 models [0] recycles 0 hard 1 soft 0 temp 1 loss 2.11 con 2.11 plddt 0.81 ptm 0.71
9 models [0] recycles 0 hard 1 soft 0 temp 1 loss 2.12 con 2.12 plddt 0.80 ptm 0.72
10 models [0] recycles 0 hard 1 soft 0 temp 1 loss 2.10 con 2.10 plddt 0.81 ptm 0.72
11 models [0] recycles 0 hard 1 soft 0 temp 1 loss 2.31 con 2.31 plddt 0.76 ptm 0.68
12 models [0] recycles 0 hard 1 soft 0 temp 1 loss 2.36 con 2.36 plddt 0.7

In [68]:
af_model.plot_pdb()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [69]:
af_model.get_seqs()

['GPPPPTPISSLGELILITLDEDNVVLIYGDAANPGSPEDAIKIPFEEFEAYLRTEYGFPKADLIIRAFEELGAEEVLINGVTKRITAITPSNPDNPIDLG']

In [70]:
af_model.save_pdb(f"tmp.pdb")

In [71]:
HTML(af_model.animate(dpi=100))